# Comprobar que todas las imágenes existen y que se pueden cargar con tensorflow

In [21]:
# Librerias

import os

import numpy as np

import tensorflow as tf

import cv2

In [22]:
# Constantes

Folder = "../zooClasificaciones_rev1-15_16-21/"

CSV = Folder + "zooClasificaciones_rev16_21_procesadas.csv"

NEW_CSV = Folder + "b01_zooClasificaciones_rev16_21_procesadas.csv"

assert (os.path.isfile(CSV)), "El csv introducido no existe"

CHANNELS = 3


In [23]:
# Extraer datos necesarios del csv (foto_id, classification, retirement, n.specific, howmanymeadian, n.n2)


#                                       FUNCION comprobarFechasImagenes

# leer excel y obtiene los datos de las fechas por camara
# Devuelve:
#   - path
#   - label

def leerCSV(inputcsv = CSV):
    # lee el csv a procesar

    # los datos que obtiene del csv
    paths, labels = np.loadtxt(inputcsv, dtype=str,
                                     delimiter=',', usecols=(0, 1), unpack=True)

    paths = [path.replace("\"","") for path in paths]
    labels = [label.replace("\"","") for label in labels]

    return paths, labels

In [24]:
# leer csv
paths, labels = leerCSV(CSV)

# data = np.column_stack([paths, labels])

In [25]:
# data

array([['procesadas/rev16/10_16/10_20220126 (1).JPG', 'HUMANORVEHICLE'],
       ['procesadas/rev16/10_16/10_20220126 (100).JPG', 'REDDEER'],
       ['procesadas/rev16/10_16/10_20220126 (1008).JPG', 'NOANIMAL'],
       ...,
       ['procesadas/rev22/8/8_20221125 (147).JPG', 'NOANIMAL'],
       ['procesadas/rev22/9/9_20220918 (448).JPG', 'NOANIMAL'],
       ['procesadas/rev22/9/9_20220919 (455).JPG', 'NOANIMAL']],
      dtype='<U45')

In [26]:
# Comprobar que existen y
# obtiene los indices de las imgs que no se pueden abrir para tensorflow y cuyo numero de canales no coincida con 3 (rgb)
def get_indexes_corruptos(files):
    indexes_imgs_corruptas = []

    # Numero de imagenes en el conjunto original
    NumeroImagenes = len(files)

    for i in range(NumeroImagenes):
        print('                                                                           ',end='\r')
        print(' Analizando: '+ str(i+1) + ' de ' + str(NumeroImagenes) + ' imágenes.',end='\r')
        # Read with OpenCV
        try:
            img =  cv2.imread(files[i])

            # Formatting the byte array to expected shapes
            img_decode = tf.io.decode_image(tf.io.read_file(files[i]), channels=3)

            if not os.path.isfile(img):
                indexes_imgs_corruptas.append(i)
            ## Canales y tamaño
            elif img.shape[2] != CHANNELS:   # Diferente numero de canales
                indexes_imgs_corruptas.append(i)

        except:
            indexes_imgs_corruptas.append(i)

    print("\nHay " + str(len(indexes_imgs_corruptas)) + " imagenes corruptas")

    return indexes_imgs_corruptas


# solo comprueba tf
def get_indexes_corruptos_tf(files):
    indexes_imgs_corruptas = []

    # Numero de imagenes en el conjunto original
    NumeroImagenes = len(files)

    for i in range(NumeroImagenes):
        print('                                                                           ',end='\r')
        print(' Analizando: '+ str(i+1) + ' de ' + str(NumeroImagenes) + ' imágenes.',end='\r')
        # Read with OpenCV
        try:
            img =  cv2.imread(files[i])

            # Formatting the byte array to expected shapes
            img_decode = tf.io.decode_image(tf.io.read_file(files[i]), channels=3)

        except:
            indexes_imgs_corruptas.append(i)

    print("\nHay " + str(len(indexes_imgs_corruptas)) + " imagenes corruptas")

    return indexes_imgs_corruptas

In [27]:
indexes_imgs_corruptas = get_indexes_corruptos(paths)

 Analizando: 290060 de 290060 imágenes.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [30]:
def write_csv(data, indexes_list, new_csv):
    assert indexes_list, "Lista de indices vacia, cuidado!"

    csvArray = np.empty([len(indexes_list), 1], dtype=object)

    for idx, index in enumerate(indexes_list):
        csvArray[idx] = data[index]

    if os.path.exists(new_csv):
        os.remove(new_csv)

    np.savetxt(new_csv, csvArray, delimiter=",",fmt='%s')

In [31]:
write_csv(paths, indexes_imgs_corruptas, NEW_CSV)